# Building interactive apps with Bokeh 

Bokeh server applications let you connect all of the powerful Python libraries for analytics and data science, such as NumPy and Pandas, to rich interactive Bokeh visualizations. Learn about Bokeh's built-in widgets, how to add them to Bokeh documents alongside plots, and how to connect everything to real python code using the Bokeh server.


## Add a single slider

Your job here is to create a single slider, use it to create a **widgetbox** layout, and then add this layout to the current document.

The slider you create here cannot be used for much, but in the later exercises, you will use it to update your plots!

In [ ]:
# Perform the necessary imports
from bokeh.io import output_notebook, show
from bokeh.layouts import widgetbox
from bokeh.models import Slider

# Create first slider: slider1
slider1 = Slider(title='slider1', start=0, end=10, step=0.1, value=2)

# Create second slider: slider2
slider2 = Slider(title='slider2', start=10, end=100, step=1, value=20)

# Add slider1 and slider2 to a widgetbox
layout = widgetbox(slider1,slider2)

# Call the output_notebook() 
output_notebook()
show(layout)



## How to combine Bokeh models into layouts

Let's begin making a Bokeh application that has a simple slider and plot, that also updates the plot based on the slider.

In this exercise, your job is to first explicitly create a ColumnDataSource. You will then combine a plot and a slider into a single column layout, and add it to the current document.

After you are done, notice how in the figure you generate, the slider will not actually update the plot, because a widget callback has not been defined. You will learn how to update the plot using widget callbacks in the next exercise.

In [ ]:
# Import the ColumnDataSource, figure class from bokeh.plotting
from bokeh.plotting import ColumnDataSource, figure

# Import row from bokeh.layouts
from bokeh.layouts import column

# Import the widgets
from bokeh.layouts import widgetbox
from bokeh.models import Slider

# Import numpy
import numpy as np

# Import functions to draw the figure in notebook
from bokeh.io import output_notebook, show


# Dataset
x = np.arange(0,10,0.1)
y = np.sin(x)

# Create ColumnDataSource: source
source = ColumnDataSource(data={'x': x, 'y': y})

# Create a figure
p = figure(x_axis_label='x values', y_axis_label='Sin(x)',
          width=400, height=300)

# Add a line to the plot
p.line('x', 'y', source=source)

# Create first slider: slider1
slider = Slider(title='slider1', start=0, end=10, step=0.1, value=2)

# Create a column layout: layout
layout = column(widgetbox(slider),p)

# Call the output_notebook() 
output_notebook()
show(layout)


## Learn about widget callbacks

You will now learn how to use widget callbacks to update the state of a Bokeh application, and in turn, the data that is presented to the user.

Your job in this exercise is to use the slider's **on_change()** function to update the plot's data from the previous example. NumPy's **sin()** function will be used to update the y-axis data of the plot.

Now that you have added a widget callback, notice how as you move the slider of your app, the figure also updates!


> **Note**

> It is currently possible to push udpates from python, to BokehJS (i.e., to update plots, etc.) using push_notebook(). It is not currently possible to get events or updates of a clean way from the other direction (e.g. to have a range or selection update trigger a python callback) without using a Bokeh Server ([reference](http://bokeh.pydata.org/en/0.11.0/docs/user_guide/server.html. Adding the capability for two-way Python<–>JS)) synchronization through Jupyter comms is a planned future addition.

> Excelent issue [here](https://groups.google.com/a/continuum.io/forum/#!topic/bokeh/YTum5CcI2y).

In [ ]:
%%writefile mybokeh.py
#write/save cell contents into mybokeh.py (use -a to append). 

# Import the ColumnDataSource, figure class from bokeh.plotting
from bokeh.plotting import ColumnDataSource, figure

# Import the widgets
from bokeh.layouts import widgetbox, column
from bokeh.models import Slider

# Import numpy
import numpy as np

# Import current document
from bokeh.io import curdoc


In [ ]:
%%writefile -a mybokeh.py
#write/save cell contents into mybokeh.py (use -a to append). 


# Define a callback function: callback
def callback(attr, old, new):

    # Read the current value of the slider: scale
    scale = slider.value

    # Compute the updated y using np.cos(scale/x): new_y
    new_y = np.sin(scale/x)
    
    # Update source with the new data values
    source.data = {'x': x, 'y': new_y}    

In [ ]:
%%writefile -a mybokeh.py

# Dataset
x = np.arange(0,10,0.1)
y = np.sin(x)

# Create ColumnDataSource: source
source = ColumnDataSource(data={'x': x, 'y': y})

# Create a figure
p = figure(x_axis_label='x values', y_axis_label='Sin(x)')

# Add a line to the plot
p.line('x', 'y', source=source)

# Create first slider: slider1
slider = Slider(title='slider1', start=0, end=10, step=0.1, value=2)


In [ ]:
%%writefile -a mybokeh.py

# Attach the callback to the 'value' property of slider
slider.on_change('value',callback)

# Create layout and add to current document
layout = column(widgetbox(slider), p)
curdoc().add_root(layout)

In [ ]:
# Open a terminal and execute
# bokeh serve mybokeh.py

# Open the browser and typed
# http://localhost:5006/mybokeh

# reference
# http://bokeh.pydata.org/en/latest/docs/user_guide/server.html

![Tabular Dataset](http://bokeh.pydata.org/en/latest/_images/bokeh_serve_client.svg)

## Proof of Concept — embedding a Bokeh server in a Notebook


[Bokeh's Author reference](https://github.com/bokeh/bokeh/blob/0.12.4/examples/howto/server_embed/notebook_embed.ipynb)

This notebook shows an apprioach to embedding Bokeh server application inside Jupyter notebook. Note that this method has some potential drawbacks, and that in future releases, there will be new APIs to mitigate these drawbacks and streamline usage.

In [ ]:
# Import the ColumnDataSource, figure class from bokeh.plotting
from bokeh.plotting import ColumnDataSource, figure

# Import the widgets
from bokeh.layouts import widgetbox, column
from bokeh.models import Slider

# Import numpy
import numpy as np


There are various application handlers that can be used to build up Bokeh documents. For example, there is a ScriptHandler that uses the code from a .py file to produce Bokeh documents. This is the handler that is used when we run bokeh serve app.py. Here we are going to use the lesser-known FunctionHandler, that gets configured with a plain Python function to build up a document.

Here is the function modify_doc(doc) that defines our app:


In [ ]:
def modify_doc(doc):
    # Dataset
    x = np.arange(0,10,0.1)
    y = np.sin(x)

    # Create ColumnDataSource: source
    source = ColumnDataSource(data={'x': x, 'y': y})

    # Create a figure
    p = figure(x_axis_label='x values', y_axis_label='Sin(x)')

    # Add a line to the plot
    p.line('x', 'y', source=source)

    # Create first slider: slider1
    slider = Slider(title='slider1', start=0, end=10, step=0.1, value=2)

    def callback(attr, old, new):
        # Read the current value of the slider: scale
        scale = slider.value

        # Compute the updated y using np.cos(scale/x): new_y
        new_y = np.sin(scale/x)
    
        # Update source with the new data values
        source.data = {'x': x, 'y': new_y} 
    
    slider.on_change('value', callback)
    
    # Create layout and add to current document
    layout = column(widgetbox(slider), p)

    doc.add_root(layout)

We take the function above and configure a FunctionHandler with it. Then we create an Application that uses handler. (It is possible, but uncommon, for Bokeh applications to have more than one handler.) The end result is that the Bokeh server will call modify_doc to build new documents for every new sessions that is opened.

In [ ]:
from bokeh.application.handlers import FunctionHandler
from bokeh.application import Application

handler = FunctionHandler(modify_doc)
app = Application(handler)

The Bokeh server runs on a Tornado IOLoop. When we run bokeh serve at the command line, an IOLoop is created and started automatically. In this case we are going to piggy-back off an existing IOLoop, the one that the Jupyter Notebook has.

In [ ]:
from tornado.ioloop import IOLoop
loop = IOLoop.current()

Below is a convenience function for showing the resulting app inline in a Jupyter notebook. In the future, there will be an API built into Bokeh that does this.


**WARNING** The function below leaks resources. It creates a new Server (with app and handler) every time it is executed, so re-executing some (or all) cells will leave stray Bokeh servers attached to the notebook IOLoop. For apps without periodioc callbacks this is probably often "OK", because the leftover server objects are effectively completely dormant when no events come in to them. But if an app has periodic callbacks, they will continue to run indefinitely, which is probably undesirable. In this case, until there is proper Bokeh API to handle cleanup, it is recommended to run the code below explicitly, and "keep track" of any servers you create. Their sessions can be shut off by executing:

```Python
server.get_sessions()[0].destroy()
```


In [ ]:
def show_app(app, notebook_url="127.0.0.1:8888"):
    from IPython.display import HTML, display
    from bokeh.embed import autoload_server
    from bokeh.server.server import Server
    
    server = Server({'/': app}, io_loop=loop, port=0, host='*', allow_websocket_origin=[notebook_url])
    server.start()
    
    script = autoload_server(model=None, url='http://127.0.0.1:%d' % server.port)
    
    display(HTML(script))

In [ ]:

show_app(app)

## Updating data sources from dropdown callbacks

You will now learn to update the plot's data using a **drop down** menu instead of a slider. This would allow users to do things like select between different data sources to view.

In [ ]:
# Perform necessary imports
from bokeh.models import ColumnDataSource, Select

# Import pandas
import pandas as pd

# Import row layout
from bokeh.layouts import row

data = pd.read_csv("fertility.csv", encoding = 'latin2')

In [ ]:
def modify_doc(doc):    
    fertility = data["fertility"]
    female_literacy = data ["female literacy"]
    population = data ["population"]

    # Create ColumnDataSource: source
    source = ColumnDataSource(data={
        'x' : fertility,
        'y' : female_literacy
    })

    # Create a new plot: plot
    plot = figure()

    # Add circles to the plot
    plot.circle('x', 'y', source=source)

    # Define a callback function: update_plot
    def update_plot(attr, old, new):
        # If the new Selection is 'female_literacy', update 'y' to female_literacy
        if new == 'female_literacy': 
            source.data = {
                'x' : fertility,
                'y' : female_literacy
            }
        # Else, update 'y' to population
        else:
            source.data = {
                'x' : fertility,
                'y' : population
            }

    # Create a dropdown Select widget: select    
    select = Select(title="distribution", options=['female_literacy', 'population'], value='female_literacy')

    # Attach the update_plot callback to the 'value' property of select
    select.on_change('value', update_plot)

    # Create layout and add to current document
    layout = row(select, plot)
    doc.add_root(layout)

In [ ]:
from bokeh.application.handlers import FunctionHandler
from bokeh.application import Application

handler = FunctionHandler(modify_doc)
app = Application(handler)

from tornado.ioloop import IOLoop
loop = IOLoop.current()

In [ ]:
def show_app(app, notebook_url="127.0.0.1:8888"):
    from IPython.display import HTML, display
    from bokeh.embed import autoload_server
    from bokeh.server.server import Server
    
    server = Server({'/': app}, io_loop=loop, port=0, host='*', allow_websocket_origin=[notebook_url])
    server.start()
    
    script = autoload_server(model=None, url='http://127.0.0.1:%d' % server.port)
    
    display(HTML(script))

In [ ]:
show_app(app)

## Synchronize two dropdowns

You will now learn how to use a dropdown callback to update another dropdown's options. This will allow you to customize your applications even further and is a powerful addition to your toolbox.

Your job in this exercise is to create two dropdown select widgets and then define a callback such that one dropdown is used to update the other dropdown.

In [ ]:
# Import the widgets
from bokeh.layouts import widgetbox

# Perform necessary imports
from bokeh.models import  Select

In [ ]:
def modify_doc(doc):    
   
    # Create two dropdown Select widgets: select1, select2
    select1 = Select(title='First', options=['A', 'B'], value='A')
    select2 = Select(title='Second', options=['1', '2', '3'], value='1')

    # Define a callback function: callback
    def callback(attr, old, new):
        # If select1 is 'A' 
        if select1.value == 'A':
            # Set select2 options to ['1', '2', '3']
            select2.options = ['1', '2', '3']

            # Set select2 value to '1'
            select2.value = '1'
        else:
            # Set select2 options to ['100', '200', '300']
            select2.options = ['100', '200', '300']

            # Set select2 value to '100'
            select2.value = '100'

    # Attach the callback to the 'value' property of select1
    select1.on_change('value', callback)

    # Create layout and add to current document
    layout = widgetbox(select1, select2)
    
    doc.add_root(layout)

In [ ]:
from bokeh.application.handlers import FunctionHandler
from bokeh.application import Application

handler = FunctionHandler(modify_doc)
app = Application(handler)

from tornado.ioloop import IOLoop
loop = IOLoop.current()

In [ ]:
def show_app(app, notebook_url="127.0.0.1:8888"):
    from IPython.display import HTML, display
    from bokeh.embed import autoload_server
    from bokeh.server.server import Server
    
    server = Server({'/': app}, io_loop=loop, port=0, host='*', allow_websocket_origin=[notebook_url])
    server.start()
    
    script = autoload_server(model=None, url='http://127.0.0.1:%d' % server.port)
    
    display(HTML(script))

In [ ]:
show_app(app)

## Button widgets

You will now gain practice with adding buttons to your interactive visualizations. Your job in this exercise is to create a button and use its on_click() method to update a plot.

In [ ]:
# Import the widgets
from bokeh.layouts import widgetbox, column

# Import the ColumnDataSource, figure class from bokeh.plotting
from bokeh.plotting import ColumnDataSource, figure

# Perform necessary imports
from bokeh.models import  Button

import numpy as np

In [ ]:
def modify_doc(doc):   
    # Dataset
    x = np.arange(0,10,0.1)
    y = np.arange(0,10,0.1)
    
    # Create ColumnDataSource: source
    source = ColumnDataSource(data={'x': x, 'y': y})
    
    # Create a new plot: plot
    plot = figure()

    # Add circles to the plot
    plot.circle('x', 'y', source=source)
    
    # Create a Button with label 'Update Data'
    button = Button(label='Update Data')

    # Define an update callback with no arguments: update
    def update():

        N = 200
        # Compute new y values: y
        y = np.sin(x) + np.random.random(len(x))

        # Update the ColumnDataSource data dictionary
        source.data = {'x': x, 'y': y}

    # Add the update callback to the button
    button.on_click(update)

    # Create layout and add to current document
    layout = column(widgetbox(button), plot)
    
    doc.add_root(layout)

In [ ]:
from bokeh.application.handlers import FunctionHandler
from bokeh.application import Application

handler = FunctionHandler(modify_doc)
app = Application(handler)

from tornado.ioloop import IOLoop
loop = IOLoop.current()

In [ ]:
def show_app(app, notebook_url="127.0.0.1:8888"):
    from IPython.display import HTML, display
    from bokeh.embed import autoload_server
    from bokeh.server.server import Server
    
    server = Server({'/': app}, io_loop=loop, port=0, host='*', allow_websocket_origin=[notebook_url])
    server.start()
    
    script = autoload_server(model=None, url='http://127.0.0.1:%d' % server.port)
    
    display(HTML(script))

In [ ]:
show_app(app)

## Button styles

You can also get really creative with your Button widgets.

In this exercise, you will practice using CheckboxGroup, RadioGroup, and Toggle to add multiple Button widgets with different styles.

In [ ]:
# Import CheckboxGroup, RadioGroup, Toggle from bokeh.models
from bokeh.models import CheckboxGroup, RadioGroup, Toggle

# Perform the necessary imports
from bokeh.io import output_notebook, show

# Add a Toggle: toggle
toggle = Toggle(button_type='success', label = 'Toggle button')

# Add a CheckboxGroup: checkbox
checkbox = CheckboxGroup(labels=['Option 1', 'Option 2', 'Option 3'])

# Add a RadioGroup: radio
radio = RadioGroup(labels=['Option 1', 'Option 2', 'Option 3'])

# Add toggle, checkbox and radio to a widgetbox
layout = widgetbox(toggle, checkbox, radio)

# Call the output_notebook() 
output_notebook()
show(layout)

